In [1]:
import pandas as pd
import re
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#Tiền xử lý dữ liệu
def tokenize_vietnamese_text(text):
    return text.split()

def filter_not_alphabet_or_number(text):
    return re.sub('\W+', ' ', text)

# Đọc dữ liệu từ file CSV
file_path = "du_lieu.csv"  # Thay "path_to_your_csv_file.csv" bằng đường dẫn tới file CSV thực tế
df = pd.read_csv(file_path)

# Lấy dữ liệu từ cột có nhãn là "Question"
text_column = df["Question"]

# Tách từng dòng của cột "Question"
x_data = []
for text in text_column:
    text = filter_not_alphabet_or_number(text)
    x_data.append(text)

# print(x_data)

In [3]:
# Đánh nhãn
y_data = df["lable"]

labeling_answer = {}
time_appear = {}
labeling_count = 0
new_y_data = []
for intent in y_data:
    if intent not in labeling_answer:
      labeling_answer[intent] = labeling_count
      labeling_count += 1
      time_appear[intent] = 0
    else:
      time_appear[intent] += 1
    new_y_data.append(labeling_answer[intent])

# print(new_y_data)
# print('labeling_answer: {}'.format(labeling_answer))
# print(time_appear)
# print(len(new_y_data))

In [4]:
input_data = x_data
output_data = new_y_data

# print(x_data[:-5])
# print(output_data[:-5])

# Encoding responses into numerical labels
labels = new_y_data

# Data preprocessing: TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(input_data)

# Word embeddings using Word2Vec
sentences = [sent.split() for sent in input_data]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

# Create word embeddings for each sentence
X_w2v = []
for sent in sentences:
    embeddings = [word2vec_model.wv[word] for word in sent if word in word2vec_model.wv]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        X_w2v.append(avg_embedding)
    else:
        # If a sentence contains no known words, add a zero vector
        X_w2v.append(np.zeros(word2vec_model.vector_size))

X_w2v = np.array(X_w2v)

# Combine TF-IDF and Word2Vec features
X_combined = np.hstack((X_tfidf.toarray(), X_w2v))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, labels, test_size=0.2, random_state=42)
print(np.array(y_train))

[ 69 148 156 ... 102 140  10]


In [5]:
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    min_child_weight=1,
    objective='multi:softmax',
    eval_metric='logloss',
    tree_method='gpu_hist'
)

# Huấn luyện mô hình và đảm bảo số lượng dữ liệu trong X_test và y_test khớp với số lượng đã sử dụng
model.fit(X_train, y_train, verbose=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [6]:
# Dự đoán nhãn trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình bằng độ chính xác và báo cáo phân loại
accuracy = accuracy_score(y_test, y_pred)
print("Độ chính xác trên tập kiểm tra:", accuracy)

classification_report_result = classification_report(y_test, y_pred)
print("Báo cáo phân loại:\n", classification_report_result)

Độ chính xác trên tập kiểm tra: 0.6848484848484848
Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.97      0.81      0.89        48
           1       0.82      0.90      0.86        60
           2       1.00      1.00      1.00         3
           3       0.44      0.47      0.45        15
           4       0.00      0.00      0.00         1
           5       0.41      0.71      0.52        17
           6       0.60      0.50      0.55         6
           7       0.56      0.42      0.48        12
           8       0.67      0.29      0.40         7
           9       0.63      0.67      0.65        18
          10       0.33      0.20      0.25         5
          11       0.33      0.22      0.27         9
          12       0.75      0.50      0.60         6
          13       0.00      0.00      0.00         0
          14       0.64      0.82      0.72        28
          15       1.00      0.67      0.80         3
          

C:\Users\baosp\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\baosp\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\baosp\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\baosp\miniconda3\envs\tf\li

In [7]:
# Get answer by id
def getAnswer(id):
  keys = [k for k, v in labeling_answer.items() if v == id]
  if len(keys) == 1:
    answer = df.loc[df['lable'] == keys[0]]
    return answer.iloc[0, 1]
  else:
    return None

# Chatbot function
def chatbot(query):
    query_vectorized_tfidf = vectorizer.transform([query])
    query_vectorized_w2v = []
    for word in query.split():
        if word in word2vec_model.wv:
            query_vectorized_w2v.append(word2vec_model.wv[word])
    if query_vectorized_w2v:
        query_vectorized_w2v = np.mean(query_vectorized_w2v, axis=0)
    else:
        # If no known words in the query, use a zero vector
        query_vectorized_w2v = np.zeros(word2vec_model.vector_size)

    query_vectorized_combined = np.hstack((query_vectorized_tfidf.toarray(), query_vectorized_w2v.reshape(1, -1)))
    response_label = model.predict(query_vectorized_combined)[0]
    response = getAnswer(response_label)
    return response

# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = chatbot(user_input)
    print("Chatbot:", response)

You:  xin chào


Chatbot: Chào bạn, trường đại học sư phạm kỹ thuật hưng yên có thể giúp gì cho bạn?


You:  Các ngành đào tạo


Chatbot: Năm 2021, nhà trường tuyển sinh 22 ngành đào tạo. các ngành này đều là những ngành hot đang được xã hội tìm kiếm nguồn nhân lực chất lượng cao đó đó cơ hội việc làm sau khi tốt nghiệp là rất lớn. nhà trường có đội ngũ giảng viên trình độ chuyên môn cao, cơ sở vật chất khang trang hiện đại đảm bảo chất lượng đào tạo tốt nhất cho sinh viên của mình. Em tham khảo thêm tại đây để xem chi tiết nhé: http://www.utehy.edu.vn/#/news-list/news-detail/fc2bd95a-0ec4-4359-a980-05cfcd310a80 


You:  exit
